## Hard-Coding the OLS estimator with NumPy and SciPy

Now we have some familarity with NumPy and SciPy, let's look at some examples of using them in practice.

The first one will be probably my 'favourite' example - in my 4th year undergrad metrics class we computed OLS by hand in MATLAB using the dataset from Mankiw, Romer and Weil. We will do this in Python. 

In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as la

## Import the data

The data are in STATA native format, so we import with Pandas (we discuss this whole thing later)

In [2]:
data = pd.read_stata('../data/mrw1992.dta')
data.head()

,c_index,c_name,c_code,cont,nonoil,inter,oecd,gdp60,gdp85,popgrowth,igdp,school
0,1,Algeria,DZA,Africa,Non-oil,Intermediate Sample,Non-OECD,2485.0,4371.0,2.6,24.100000,4.5
1,2,Angola,AGO,Africa,Non-oil,Not intermediate sample,Non-OECD,1588.0,1171.0,2.1,5.800000,1.8
2,3,Benin,BEN,Africa,Non-oil,Not intermediate sample,Non-OECD,1116.0,1071.0,2.4,10.800000,1.8
3,4,Botswana,BWA,Africa,Non-oil,Intermediate Sample,Non-OECD,959.0,3671.0,3.2,28.299999,2.9
4,5,Burkina Faso,BFA,Africa,Non-oil,Not intermediate sample,Non-OECD,529.0,857.0,0.9,12.700000,0.4


Question: Why couldnt we import that as a NumPy array?

In [3]:
# subset OECD data
oecd = data[data['oecd']=='OECD']
oecd.head()

,c_index,c_name,c_code,cont,nonoil,inter,oecd,gdp60,gdp85,popgrowth,igdp,school
52,53,Japan,JPN,Asia,Non-oil,Intermediate Sample,OECD,3493.0,13893.0,1.2,36.000000,10.9
69,70,Austria,AUT,Europe,Non-oil,Intermediate Sample,OECD,5939.0,13327.0,0.4,23.400000,8.0
70,71,Belgium,BEL,Europe,Non-oil,Intermediate Sample,OECD,6789.0,14290.0,0.5,23.400000,9.3
72,73,Denmark,DNK,Europe,Non-oil,Intermediate Sample,OECD,8551.0,16491.0,0.6,26.600000,10.7
73,74,Finland,FIN,Europe,Non-oil,Intermediate Sample,OECD,6527.0,13779.0,0.7,36.900002,11.5


Now we want to generate some dependent and independent variables...

* MRW use 'growth' as the dependent variable
* Independent variables are 'popgrowth', 'igdp', 'school' and 'lngdp60'

In [4]:
# extract the dependent variable

gdp_data = oecd[['gdp60', 'gdp85']].values

y = (np.log(gdp_data[:,1]) - np.log(gdp_data[:,0])) / 25

In [5]:
print(y)

[ 0.05522499  0.03233005  0.02977024  0.02627068  0.02988792  0.02934746
  0.02748344  0.04451345  0.02705376  0.0325375   0.02154728  0.03640495
  0.03767366  0.03867298  0.02677387  0.01728809  0.02680057  0.02229919
  0.02223881  0.0171672   0.01851776  0.01026157]


In [6]:
expl_var = oecd[['popgrowth', 'igdp', 'school']].values

In [7]:
nObs = np.shape(y)[0]
iota = np.ones(nObs).reshape((nObs,1))

In [8]:
iota.shape

(22, 1)

In [9]:
log_gdp60 = np.log(gdp_data[:,0]).reshape((nObs,1))
log_gdp60.shape

(22, 1)

In [10]:
X = np.hstack(
                      tup = (iota, log_gdp60, expl_var)
                    )

In [11]:
print(X)

[[  1.           8.15851593   1.20000005  36.          10.89999962]
 [  1.           8.68929577   0.40000001  23.39999962   8.        ]
 [  1.           8.82305908   0.5         23.39999962   9.30000019]
 [  1.           9.05380344   0.60000002  26.60000038  10.69999981]
 [  1.           8.78370285   0.69999999  36.90000153  11.5       ]
 [  1.           8.88391781   1.          26.20000076   8.89999962]
 [  1.           8.94832611   0.5         28.5          8.39999962]
 [  1.           7.72179174   0.69999999  29.29999924   7.9000001 ]
 [  1.           8.39185715   1.10000002  25.89999962  11.39999962]
 [  1.           8.49964046   0.60000002  24.89999962   7.0999999 ]
 [  1.           8.94754601   1.39999998  25.79999924  10.69999981]
 [  1.           8.97941685   0.69999999  29.10000038  10.        ]
 [  1.           7.72841597   0.60000002  22.5          5.80000019]
 [  1.           8.23376846   1.          17.70000076   8.        ]
 [  1.           8.96213531   0.40000001  24.5  

## Matrix Algebra

In [12]:
# matrix multiplication
XX = X.T @ X

In [13]:
XX_2 = (X.T).dot(X)

In [16]:
np.array_equal(XX, XX_2)

True

In [17]:
XX == XX_2

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]], dtype=bool)

In [20]:
# inverse
XX_inv = la.pinv2(XX)
print(XX_inv)

[[  1.67827326e+01  -1.87205955e+00  -2.80562374e-01  -6.07912750e-02
    1.55817859e-01]
 [ -1.87205955e+00   2.35814178e-01   2.04860900e-02   2.13578604e-03
   -2.83056002e-02]
 [ -2.80562374e-01   2.04860900e-02   1.38693770e-01   2.43492635e-03
   -1.10700566e-02]
 [ -6.07912750e-02   2.13578604e-03   2.43492635e-03   2.08444957e-03
   -1.54344685e-03]
 [  1.55817859e-01  -2.83056002e-02  -1.10700566e-02  -1.54344685e-03
    1.55891095e-02]]


In [24]:
# Does (X'X)^-1 X'X = I -> don't do this
print(np.round(XX_inv @ XX, decimals=2).astype(int))

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [31]:
# do this
np.allclose(XX_inv @ XX, np.eye(5), atol = 1e-8)

True

## Compute the OLS estimator & VCOV Matrix

In [53]:
def compute_OLS(dependent_var, explanatory_var):
    """
        Compute the OLS estimator beta_hat
        
        beta_hat = (X'X)^-1 X'y
        
        INPUTS:
            * dependent_var: 1-D numpy array
            * explanatory_var: 2-D numpy array
            
        OUTPUT:
            * beta_hat: least-squares coefficients
            
        EXAMPLE: (OPTIONAL)    
    """
    import numpy
    from scipy import linalg
    
    # check inputs
    assert type(dependent_var) == numpy.ndarray
    assert type(explanatory_var) == numpy.ndarray
    
    # redefine variables locally
    X = explanatory_var
    y = dependent_var
    
    # 
    assert y.ndim == 1
    assert X.ndim == 2
    
    # 
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert y_rows == X_rows
    assert X_rows >= X_cols
    
    # compute beta_hat
    XX = X.T.dot(X)
    Xy = X.T.dot(y)
    
    beta_hat = linalg.solve(XX, Xy)
    
    ##
    n_coeff = beta_hat.size
    assert n_coeff == X_cols
    
    return beta_hat

In [54]:
compute_OLS(y,X)

array([ 0.14793942, -0.01556042, -0.00569366,  0.00052743,  0.00091834])

In [68]:
def get_residuals(dep_var, expl_var):
    """
        YOU SHOULD WRITE DOCs
    """
    import numpy
    from scipy import linalg
    
    assert type(dep_var) == numpy.ndarray
    assert type(expl_var) == numpy.ndarray
    
    X = expl_var
    y = dep_var 
    
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    
    ## get projection matrix
    XX_inv = linalg.pinv2(X.T.dot(X))
    P = X.dot(XX_inv).dot(X.T)
    
    ##
    P_rows, P_cols = P.shape
    assert P_rows == P_cols
    assert X_rows == P_rows
    
    M = numpy.eye(P_rows) - P
    
    residuals = M.dot(y)
    
    return residuals

In [69]:
get_residuals(y,X)

array([ 0.0120704 ,  0.00218853,  0.00108566, -0.00122752, -0.00741102,
        0.00334721, -0.00111534, -0.00199476, -0.0081714 ,  0.00061894,
       -0.00262745,  0.00764289, -0.00378579,  0.00786582,  0.0003895 ,
       -0.00238042, -0.00233864, -0.00269425,  0.00741865,  0.00232742,
       -0.00297055, -0.00823786])

In [71]:
def compute_sigma2(dep_var, expl_var):
    """
        WRITE DOCS
    """
    
    import numpy
    from scipy import linalg
    
    # type checking
    assert type(dep_var) == numpy.ndarray
    assert type(expl_var) == numpy.ndarray
    
    X = expl_var
    y = dep_var 
    
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    
    # now get the residuals
    u = get_residuals(y, X)
    uu = u.dot(u)
    
    # now sigma2
    df = X_rows - X_cols
    sigma2 = uu / df
    
    assert numpy.isscalar(sigma2)
    
    return sigma2
    

In [72]:
compute_sigma2(y,X)

3.464008691415308e-05

In [84]:
def compute_vcov_homosk(dep_var, expl_var):
    """
        WRITE DOCS
    """
    
    import numpy
    from scipy import linalg
    
    # type checking
    assert type(dep_var) == numpy.ndarray
    assert type(expl_var) == numpy.ndarray
    print(expl_var.shape)
    X = expl_var
    y = dep_var 
    print(X.shape)
    X_rows, X_cols = X.shape
    y_rows = y.size
    
    assert X_rows == y_rows
    
    # compute vcov matrix
    sigma2 = compute_sigma2(y,X)
    
    XX = X.T.dot(X)
    XX_inv = linalg.pinv2(XX)
    
    vcov_mat = sigma2 * XX_inv
    print(vcov_mat.shape)
    
    # get std Errors
    std_errors = numpy.sqrt(numpy.diag(vcov_mat))
    
    return [std_errors, vcov_mat]

In [85]:
std_errors, vcov = compute_vcov_homosk(y, X)

(22, 5)
(22, 5)
(5, 5)


In [86]:
print(std_errors)

[ 0.02411131  0.00285808  0.00219189  0.00026871  0.00073485]


In [76]:
vcov

array([[  1.31436509e-05,  -4.47538397e-08,  -1.30151795e-06],
       [ -4.47665052e-08,   1.07927633e-07,  -2.89801790e-07],
       [ -1.30148157e-06,  -2.89803125e-07,   9.85303700e-07]], dtype=float32)

In [81]:
X.shape

(22, 5)